In [8]:
import sys
sys.version

'3.6.7 |Anaconda, Inc.| (default, Oct 28 2018, 19:44:12) [MSC v.1915 64 bit (AMD64)]'

In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

In [4]:
data = pd.read_csv('./data/train_V2.csv')
train, test = train_test_split(data, train_size = 0.7)

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3112876 entries, 4307427 to 4257903
Data columns (total 29 columns):
Id                 object
groupId            object
matchId            object
assists            int64
boosts             int64
damageDealt        float64
DBNOs              int64
headshotKills      int64
heals              int64
killPlace          int64
killPoints         int64
kills              int64
killStreaks        int64
longestKill        float64
matchDuration      int64
matchType          object
maxPlace           int64
numGroups          int64
rankPoints         int64
revives            int64
rideDistance       float64
roadKills          int64
swimDistance       float64
teamKills          int64
vehicleDestroys    int64
walkDistance       float64
weaponsAcquired    int64
winPoints          int64
winPlacePerc       float64
dtypes: float64(6), int64(19), object(4)
memory usage: 712.5+ MB


In [5]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1334090 entries, 1753135 to 2769105
Data columns (total 29 columns):
Id                 1334090 non-null object
groupId            1334090 non-null object
matchId            1334090 non-null object
assists            1334090 non-null int64
boosts             1334090 non-null int64
damageDealt        1334090 non-null float64
DBNOs              1334090 non-null int64
headshotKills      1334090 non-null int64
heals              1334090 non-null int64
killPlace          1334090 non-null int64
killPoints         1334090 non-null int64
kills              1334090 non-null int64
killStreaks        1334090 non-null int64
longestKill        1334090 non-null float64
matchDuration      1334090 non-null int64
matchType          1334090 non-null object
maxPlace           1334090 non-null int64
numGroups          1334090 non-null int64
rankPoints         1334090 non-null int64
revives            1334090 non-null int64
rideDistance       1334090 non-nu

In [5]:
# 회귀분석을 하기 위해 문자열을 범주값으로 변환
matchTyp = {'squad-fpp': 0, 'duo': 1, 'solo-fpp': 2, 'squad': 3, 'duo-fpp': 4, 'solo': 5,
       'normal-squad-fpp': 6, 'crashfpp': 7, 'flaretpp': 8, 'normal-solo-fpp': 9,
       'flarefpp': 10, 'normal-duo-fpp': 11, 'normal-duo': 12, 'normal-squad': 13,
       'crashtpp': 14, 'normal-solo': 15 }

train['matchType'] = train['matchType'].replace(matchTyp)
test['matchType'] = test['matchType'].replace(matchTyp)

In [7]:
# null값 확인 및 na가 포함된 행 제거
# inplace=False로 하면 기존 혹은 새로운 변수에 할당해야 하고, inplace = True 하면 해당변수에 적용됨
# na가 포함된 행을 제거하는 것은 데이터 소실이 크기 때문에 inplace=False가 default
train.dropna(inplace = True)
print(train.isnull().any().any())
test.dropna(inplace = True)
print(test.isnull().any().any())

False
False


In [8]:
train_df = train.drop(['Id','groupId','matchId'], axis = 1)

In [9]:
test_df = test.drop(['Id','groupId','matchId'], axis = 1)

### 데이터 전 처리 없이 multi linear regression

###### learning rate = 0.01

In [10]:
train_np = np.array(train_df)
test_np = np.array(test_df)

x_train = train_np[:,0:-1] # id, groupId, matchid 제외
y_train = train_np[:, [-1]]
print('pass')

x_test = test_np[:,:-1] # id, groupId, matchid 제외
y_test = test_np[:,[-1]]
print('pass')

# placeholders for a tensor that will be always fed.
X = tf.placeholder(tf.float32, shape=[None, 25]) # 총 29개 칼럼에서 3개 빼면 26, y값 까지 빼면 25
Y = tf.placeholder(tf.float32, shape=[None, 1])

W = tf.Variable(tf.random_normal([25, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

# Hypothesis(model)
hypothesis = tf.matmul(X, W) + b

# Simplified cost/loss function
cost = tf.reduce_mean(tf.square(hypothesis - Y))

# Minimize
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.000000000001) # 0.01로 할 시에 발산..
train = optimizer.minimize(cost)

# Launch the graph in a session.
with tf.Session() as sess:
    # Initializes global variables in the graph.
    sess.run(tf.global_variables_initializer())

    for step in range(51):
        cost_val, hy_val, _ = sess.run(
            [cost, hypothesis, train], feed_dict={X: x_train, Y: y_train})
        if step % 5 == 0:
            print(step, "Cost: ", cost_val, "\nPrediction:\n", hy_val)
    Weight = sess.run(W)
    print(Weight)
    y_predict = sess.run(hypothesis, feed_dict={X:x_test})
    print(y_predict)
    
# 계속 발산 문제가 발생하는데, 단위가 다른 killpoint, rankpoint 때문이거나, learning rate가 높거나

pass
pass
0 Cost:  3192517.0 
Prediction:
 [[1871.8335 ]
 [-186.7072 ]
 [1588.5277 ]
 ...
 [ 593.3451 ]
 [ 825.23926]
 [-188.75899]]
5 Cost:  3192194.0 
Prediction:
 [[1871.75   ]
 [-186.75667]
 [1588.4492 ]
 ...
 [ 593.22424]
 [ 825.1378 ]
 [-188.79408]]
10 Cost:  3191866.8 
Prediction:
 [[1871.6665 ]
 [-186.80603]
 [1588.371  ]
 ...
 [ 593.10345]
 [ 825.0363 ]
 [-188.82912]]
15 Cost:  3191539.0 
Prediction:
 [[1871.5829 ]
 [-186.85538]
 [1588.2927 ]
 ...
 [ 592.9827 ]
 [ 824.93494]
 [-188.86415]]
20 Cost:  3191216.5 
Prediction:
 [[1871.4995 ]
 [-186.90475]
 [1588.2144 ]
 ...
 [ 592.8619 ]
 [ 824.83356]
 [-188.89932]]
25 Cost:  3190893.0 
Prediction:
 [[1871.416  ]
 [-186.95396]
 [1588.1359 ]
 ...
 [ 592.74115]
 [ 824.7319 ]
 [-188.93423]]
30 Cost:  3190568.2 
Prediction:
 [[1871.3325 ]
 [-187.00331]
 [1588.0575 ]
 ...
 [ 592.6204 ]
 [ 824.6305 ]
 [-188.9693 ]]
35 Cost:  3190242.2 
Prediction:
 [[1871.249  ]
 [-187.05263]
 [1587.9791 ]
 ...
 [ 592.4997 ]
 [ 824.5291 ]
 [-189.00436]]


In [14]:
test_id = pd.DataFrame(test['Id'])
test_winPlacePerc = pd.DataFrame(test['winPlacePerc'])
y_predict_df = pd.DataFrame(y_predict,index = test_id.index,columns=['predict'])
diff = pd.DataFrame(test['winPlacePerc']-y_predict_df['predict'],
                   columns=['diff'])

mae = sum(abs(diff['diff']))/diff['diff'].count() # Mean absolute error

result = pd.DataFrame({'Id':test_id['Id'],'winPlacePerc':test_winPlacePerc['winPlacePerc'], 
                       'predict':y_predict_df['predict'],'diff':diff['diff'],'MAE':mae})
result.to_csv('./result/Multi_Linear_Regression_all_inputs.csv',sep=',', encoding='utf-8')
print(result.head())
print('Mae\t : ', mae)

                     Id  winPlacePerc      predict         diff         MAE
399423   b9afa39d984a72        0.6735  -184.119568   184.793068  1465.89581
254232   258779308864c1        0.1852  2131.218994 -2131.033794  1465.89581
140019   0953bdd466ba40        0.2000    88.885963   -88.685963  1465.89581
1461802  50abe5de009ba5        0.2500  1891.363403 -1891.113403  1465.89581
4301593  e7245481830af5        0.0000  -221.331985   221.331985  1465.89581
Mae	 :  1465.895809973223


### 문제
> - 예측변수인 winPlacePerc는 0~1 사이의 값인데 반해, x변수값 중에서는 단위가 큰 값이 존재하므로, 단위의 normalization or MinMaxSaler 함수를 적용해야 함